In [21]:
# install the latest version of the openai library
!pip install openai -q --upgrade

# imports
from openai import OpenAI
import json
import time
import requests



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [22]:
# Load the OpenAI key from a file
with open('../secrets/openAI_key', 'r') as file:
    openAI_key = file.read().replace('\n', '')

# load the github token
with open('../secrets/github_token', 'r') as file:
    github_token = file.read().replace('\n', '')

In [23]:
# initalize the client
client = OpenAI(
    api_key = openAI_key
)

### Function calling with chat completion API

In [24]:
# Get repository structure

def get_repo_structure(url, branch=None, relativePaths=None):
    # URL of Askthecode API endpoint
    get_repo_structure_url = "https://gabriel.askthecode.ai/api/repository/structure"

    # parameters: url(required), branch(optional), relativePaths(optional)
    params = {
        'url': url,
        'branch': branch,
        'relativePaths': relativePaths
    }

    # headers
    headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {github_token}", 
    "Content-Type": "application/json"
    }

    # Make the post request
    response = requests.post(get_repo_structure_url, json=params, headers=headers)

    # check if the response is successful
    if response.status_code == 200:
        # Parsing the response JSON
        structure_response = response.json()
        # Return the response data instead of printing
        return structure_response
    
    else:
        print(f"Failed to get the repository structure: {response.status_code}")


# test the function
test_repo_structure_url = get_repo_structure(url="https://github.com/recommenders-team/recommenders")

In [25]:
available_functions = {
    "get_repo_structure": get_repo_structure
}

In [26]:
# GPT specifications of functions

functions = [
    {
        "name": "get_repo_structure",
        "description": "Retrieves the Github repository file structure to analyze it and be able to query only relevant files. If the provided URL contains specific branch and directory information, prioritize using that over querying the entire repository structure.",
        "parameters": {
            "type": "object",
            "properties": {
                "url": {
                    "minLength": 1,
                    "type": "string",
                    "description": "Full Github repository URL provided by the user. For example: https://github.com/[owner]/[repo]/blob/[branch]/[file-path]#[additional-parameters]. The URL MUST be identical to the one, that was provided by the user, you MUST NEVER alter or truncate it. This is crucial for valid responses. You should NEVER truncate additional-parameters.",
                },
                "branch": {
                    "type": "string",
                    "description": "Repository branch. Provide only if user has explicitly specified it or the previous plugin response contains it.",
                    "nullable": True
                },
                "relativePaths": {
                    "type": "array",
                    "items": {
                        "type": "string"
                    },
                    "description": "Relative paths to retrieve. USE only paths you are certain that exist, NEVER invent them. If the provided URL contains a specific directory path, extract and use it. Otherwise, this should be a directory path or pattern only. Patterns accept * symbol as 'any substring'",
                    "nullable": True
                }
        },
        "required": ["url"],
        "aditinalProperties": False
    }
}

]

In [27]:
# pass functions to chatcompletionAPI

def get_gpt_response(messages):
    chat_completion = client.chat.completions.create(
        model = "gpt-3.5-turbo-1106",
        messages = messages,
        functions = functions,
        function_call = "auto"
    )
    return chat_completion

# execute the function

def execute_function_call(function_name,arguments):
    function = available_functions.get(function_name,None)
    if function:
        arguments = json.loads(arguments)
        results = function(**arguments)
    else:
        results = f"Error: function {function_name} does not exist"
    return results
    
    


In [28]:
# test the function

#query = "Tell me a joke"
#query = "can you analyse the structure of a repository?"
#query = "Get the structure of the following repository: https://github.com/recommenders-team/recommenders"
query = "tell me about this repository: https://github.com/recommenders-team/recommenders"

messages = [
    {
        "role": "user",
        "content": query
    }
]
response = get_gpt_response(messages)
print(response.choices[0].message)

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"url":"https://github.com/recommenders-team/recommenders"}', name='get_repo_structure'), tool_calls=None)


In [29]:
function_name = response.choices[0].message.function_call.name # check what function is being called by GPT
arguments = response.choices[0].message.function_call.arguments # check the arguments being passed to the function
print(function_name)
print(arguments)

get_repo_structure
{"url":"https://github.com/recommenders-team/recommenders"}


In [32]:
# execute the function call 
function_response = execute_function_call(function_name,arguments) 
print(function_response)



{'branchName': 'main', 'files': ['.devcontainer/Dockerfile', '.devcontainer/devcontainer.json', '.github/.codecov.yml', '.github/CODEOWNERS', '.github/ISSUE_TEMPLATE.md', '.github/ISSUE_TEMPLATE/bug_report.md', '.github/ISSUE_TEMPLATE/feature_request.md', '.github/ISSUE_TEMPLATE/general-ask.md', '.github/PULL_REQUEST_TEMPLATE.md', '.github/actions/azureml-test/action.yml', '.github/actions/get-test-groups/action.yml', '.github/workflows/azureml-cpu-nightly.yml', '.github/workflows/azureml-gpu-nightly.yml', '.github/workflows/azureml-release-pipeline.yml', '.github/workflows/azureml-spark-nightly.yml', '.github/workflows/azureml-unit-tests.yml', '.github/workflows/sarplus.yml', '.github/workflows/update_documentation.yml', 'AUTHORS.md', 'CODE_OF_CONDUCT.md', 'CONTRIBUTING.md', 'GLOSSARY.md', 'LICENSE', 'MANIFEST.in', 'NEWS.md', 'README.md', 'SECURITY.md', 'SETUP.md', 'contrib/README.md', 'contrib/azureml_designer_modules/README.md', 'contrib/azureml_designer_modules/entries/map_entry.py

In [34]:
# add the function response to the messages
messages.append(response.choices[0].message) # insert the response related to the function call
messages.append(
    {
    "role" : "function",
    "name" : function_name,
    "content" : str(function_response)
    }
)
messages

[{'role': 'user',
  'content': 'tell me about this repository: https://github.com/recommenders-team/recommenders'},
 ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"url":"https://github.com/recommenders-team/recommenders"}', name='get_repo_structure'), tool_calls=None),
 {'role': 'function',
  'name': 'get_repo_structure',
  'content': "{'branchName': 'main', 'files': ['.devcontainer/Dockerfile', '.devcontainer/devcontainer.json', '.github/.codecov.yml', '.github/CODEOWNERS', '.github/ISSUE_TEMPLATE.md', '.github/ISSUE_TEMPLATE/bug_report.md', '.github/ISSUE_TEMPLATE/feature_request.md', '.github/ISSUE_TEMPLATE/general-ask.md', '.github/PULL_REQUEST_TEMPLATE.md', '.github/actions/azureml-test/action.yml', '.github/actions/get-test-groups/action.yml', '.github/workflows/azureml-cpu-nightly.yml', '.github/workflows/azureml-gpu-nightly.yml', '.github/workflows/azureml-release-pipeline.yml', '.github/workflows/azureml-spark-nightly.yml', '.gith

In [35]:
response = get_gpt_response(messages)
print(response.choices[0].message.content)

The repository "recommenders" contains a wide variety of files related to collaborative filtering, content-based filtering, and other recommendation systems. It includes code for model development, evaluation, tuning, datasets, and utility functions. It also has directories for examples, documentation, tests, tools, and more.

Here are some of the key files and directories in the repository:
- .devcontainer: Development container configuration files
- .github: GitHub-related configuration and templates
- AUTHORS.md: List of authors contributing to the project
- CODE_OF_CONDUCT.md: Code of conduct for contributors
- LICENSE: Project license file
- docs: Documentation files and configuration
- examples: Example notebooks and scripts for quick start, model building, evaluations, and more
- recommenders: Core codebase for developing recommendation models
- scenarios: Directory containing specific recommendation scenarios like ads, food_and_restaurants, gaming, news, retail, travel
- tests: